In [5]:
import boto3

s3 = boto3.client('s3')

s3 = boto3.resource(

        service_name = 's3',
        region_name = 'us-east-2',
        aws_access_key_id = aws_access_key_id,
        aws_secret_access_key = aws_secret_access_key
    
)

for object in s3.Bucket('newbucketformyproject').objects.all():
    print(object.key)


IDS_mapping.csv
diabetes_012_health_indicators_BRFSS2015.csv
diabetes_binary_5050split_health_indicators_BRFSS2015.csv
diabetes_binary_health_indicators_BRFSS2015.csv
diabetic_data.csv
drugsComTest_raw.tsv
drugsComTrain_raw.tsv


In [7]:
# Set the Hadoop configurations for S3 access
spark._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark._jsc.hadoopConfiguration().set("fs.s3a.access.key", aws_access_key_id)
spark._jsc.hadoopConfiguration().set("fs.s3a.secret.key", aws_secret_access_key)


In [9]:
diabetic_data = spark.read.csv("s3a://newbucketformyproject/diabetic_data.csv", header=True)

In [10]:
diabetic_data

DataFrame[encounter_id: string, patient_nbr: string, race: string, gender: string, age: string, weight: string, admission_type_id: string, discharge_disposition_id: string, admission_source_id: string, time_in_hospital: string, payer_code: string, medical_specialty: string, num_lab_procedures: string, num_procedures: string, num_medications: string, number_outpatient: string, number_emergency: string, number_inpatient: string, diag_1: string, diag_2: string, diag_3: string, number_diagnoses: string, max_glu_serum: string, A1Cresult: string, metformin: string, repaglinide: string, nateglinide: string, chlorpropamide: string, glimepiride: string, acetohexamide: string, glipizide: string, glyburide: string, tolbutamide: string, pioglitazone: string, rosiglitazone: string, acarbose: string, miglitol: string, troglitazone: string, tolazamide: string, examide: string, citoglipton: string, insulin: string, glyburide-metformin: string, glipizide-metformin: string, glimepiride-pioglitazone: str

In [13]:
IDS_mapping = spark.read.csv("s3a://newbucketformyproject/IDS_mapping.csv", header=True)
IDS_mapping

DataFrame[admission_type_id: string, description: string]

In [15]:
IDS_mapping

DataFrame[admission_type_id: string, description: string]

In [17]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml import Pipeline

# Initialize Spark session
spark = SparkSession.builder.appName("DiabetesAnalysis").getOrCreate()

# Load data
diabetic_data = spark.read.csv("s3a://newbucketformyproject/diabetic_data.csv", header=True)

# Convert string columns to numeric
diabetic_data_clean = diabetic_data.withColumn("time_in_hospital", col("time_in_hospital").cast(IntegerType()))
diabetic_data_clean = diabetic_data_clean.withColumn("num_lab_procedures", col("num_lab_procedures").cast(IntegerType()))
diabetic_data_clean = diabetic_data_clean.withColumn("num_procedures", col("num_procedures").cast(IntegerType()))
diabetic_data_clean = diabetic_data_clean.withColumn("num_medications", col("num_medications").cast(IntegerType()))

# Convert categorical features to numerical using StringIndexer
categorical_columns = ['race', 'gender', 'age', 'admission_type_id', 'discharge_disposition_id', 'admission_source_id', 
                        'payer_code', 'medical_specialty', 'diag_1', 'diag_2', 'diag_3', 'max_glu_serum', 'A1Cresult', 
                        'metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 
                        'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 
                        'troglitazone', 'tolazamide', 'examide', 'citoglipton', 'insulin', 'glyburide-metformin', 
                        'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 'metformin-pioglitazone', 
                        'change', 'diabetesMed']

indexers = [StringIndexer(inputCol=col, outputCol=col + "_index") for col in categorical_columns]
pipeline = Pipeline(stages=indexers)
model = pipeline.fit(diabetic_data_clean)
diabetic_data_indexed = model.transform(diabetic_data_clean)

# Assemble features
assembler = VectorAssembler(inputCols=[col + "_index" for col in categorical_columns] + 
                                      ['time_in_hospital', 'num_lab_procedures', 'num_procedures', 'num_medications'], 
                             outputCol='features')
diabetic_data_prepared = assembler.transform(diabetic_data_indexed)


In [23]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Step 1: Convert 'readmitted' column to numeric index
indexer = StringIndexer(inputCol='readmitted', outputCol='readmitted_index')
diabetic_data_indexed = indexer.fit(diabetic_data_prepared).transform(diabetic_data_prepared)

# Step 2: Split the data into training and testing sets
(training_data, test_data) = diabetic_data_indexed.randomSplit([0.7, 0.3], seed=1234)

# Step 3: Initialize the Logistic Regression model
logistic_regression = LogisticRegression(labelCol='readmitted_index')

# Step 4: Train the model on the training data
logistic_regression_model = logistic_regression.fit(training_data)

# Step 5: Make predictions on the test data
predictions = logistic_regression_model.transform(test_data)

# Step 6: Initialize the evaluator
evaluator = MulticlassClassificationEvaluator(
    labelCol='readmitted_index',
    predictionCol='prediction',
    metricName='accuracy'
)

# Step 7: Evaluate the model's accuracy
accuracy = evaluator.evaluate(predictions)
print(f"Logistic Regression Accuracy: {accuracy:.4f}")


Logistic Regression Accuracy: 0.5448


In [29]:
from pyspark.ml.classification import DecisionTreeClassifier

# Increase maxBins to handle categorical features with many values
dt = DecisionTreeClassifier(labelCol='readmitted_index', featuresCol='features', maxBins=1000)
dt_model = dt.fit(trainingData)

# Make predictions and evaluate
dt_predictions = dt_model.transform(testData)
accuracy = evaluator.evaluate(dt_predictions)
print(f"Decision Tree Accuracy: {accuracy:.4f}")


Decision Tree Accuracy: 0.5404


In [31]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder

# Index categorical features
indexers = [
    StringIndexer(inputCol=column, outputCol=column + "_index")
    for column in ['categorical_feature1', 'categorical_feature2']  # Replace with actual column names
]

# Apply StringIndexer
pipeline = Pipeline(stages=indexers)
model = pipeline.fit(trainingData)
indexed_df = model.transform(trainingData)

# One-Hot Encode the indexed features
encoder = OneHotEncoder(
    inputCols=['categorical_feature1_index', 'categorical_feature2_index'],  # Replace with actual column names
    outputCols=['categorical_feature1_vec', 'categorical_feature2_vec']
)
indexed_df = encoder.fit(indexed_df).transform(indexed_df)


Py4JJavaError: An error occurred while calling o4922.fit.
: org.apache.spark.SparkException: Input column categorical_feature1 does not exist.
	at org.apache.spark.ml.feature.StringIndexerBase.$anonfun$validateAndTransformSchema$2(StringIndexer.scala:128)
	at scala.collection.TraversableLike.$anonfun$flatMap$1(TraversableLike.scala:293)
	at scala.collection.IndexedSeqOptimized.foreach(IndexedSeqOptimized.scala:36)
	at scala.collection.IndexedSeqOptimized.foreach$(IndexedSeqOptimized.scala:33)
	at scala.collection.mutable.ArrayOps$ofRef.foreach(ArrayOps.scala:198)
	at scala.collection.TraversableLike.flatMap(TraversableLike.scala:293)
	at scala.collection.TraversableLike.flatMap$(TraversableLike.scala:290)
	at scala.collection.mutable.ArrayOps$ofRef.flatMap(ArrayOps.scala:198)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema(StringIndexer.scala:123)
	at org.apache.spark.ml.feature.StringIndexerBase.validateAndTransformSchema$(StringIndexer.scala:115